Parte A: Pandas

In [1]:
!pip show pandas

In [2]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 36.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 46.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 KB 37.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pandas as pd
import time

Carga del dataset "ratings" y "movies"

In [4]:
peliculas=pd.read_csv('movies.csv')
clasificacion=pd.read_csv('ratings.csv')

Contar tipos de valoraciones

In [7]:
tiempo_inicio=time.time()

count_clasificacion=clasificacion['rating'].value_counts().sort_index()

print("Valoraciones y las frecuencias")
print(count_clasificacion)

tiempo_fin=time.time()
print(f"Tiempo tomado: {tiempo_fin - tiempo_inicio: .2f} segundos")

Valoraciones y las frecuencias
rating
0.5     566306
1.0    1013645
1.5     562409
2.0    2146492
2.5    1760733
3.0    6400664
3.5    4465001
4.0    8835955
4.5    3123055
5.0    4957902
Name: count, dtype: int64
Tiempo tomado:  0.22 segundos


Calcular valoracion media, además de guardar un nuevo archivo

In [9]:
tiempo_inicio=time.time()

clasificacion_promedio=clasificacion.groupby('movieId')['rating'].mean()

peliculas=peliculas.set_index('movieId')
peliculas['clasificacion_promedio']=clasificacion_promedio
peliculas.reset_index(inplace=True)

peliculas.to_csv('clasificacion_peliculas.csv', index=False)

tiempo_fin=time.time()
print(f"Tiempo tomado: {tiempo_fin-tiempo_inicio:.2f} segundos")

Tiempo tomado: 0.70 segundos


Parte B: Spark

In [10]:
!pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: py4j
Required-by: 


In [11]:
from pyspark.sql import SparkSession
import time

In [14]:
spark=SparkSession.builder \
    .appName("Movielens Analysis") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [16]:
peliculas=spark.read.csv('hdfs://main:9000/tmp/movielens/movies.csv', header=True, inferSchema=True)
clasificacion=spark.read.csv('hdfs://main:9000/tmp/movielens/ratings.csv', header=True, inferSchema=True)

Contar tipos de valoraciones

In [18]:
tiempo_inicio=time.time()
recuentos_calificacion=clasificacion.groupBy('rating').count().orderBy('rating')
recuentos_calificacion.show()
tiempo_fin=time.time()
print(f"Tiempo transcurrido:{tiempo_fin-tiempo_inicio:.2f} segundos")

+------+-------+
|rating|  count|
+------+-------+
|   0.5| 566306|
|   1.0|1013645|
|   1.5| 562409|
|   2.0|2146492|
|   2.5|1760733|
|   3.0|6400664|
|   3.5|4465001|
|   4.0|8835955|
|   4.5|3123055|
|   5.0|4957902|
+------+-------+

Tiempo transcurrido:2.15 segundos


Calculo valoracion media y guardado de nueov archivo

In [19]:
tiempo_inicio=time.time()
clasificacion_promedio=clasificacion.groupBy('movieId').avg('rating').withColumnRenamed('avg(rating)','average_rating')
peliculas_con_clasificacion=peliculas.join(clasificacion_promedio, 'movieId', 'left')
peliculas_con_clasificacion.write.csv('peliculas_clasificacion.csv', header=True)
fin_tiempo=time.time()
print(f"Tiempo trascurrido:{fin_tiempo-tiempo_inicio:.2f} segundos")

Tiempo trascurrido:3.74 segundos
